# Download packages

In [3]:
%pip install tensorly

Note: you may need to restart the kernel to use updated packages.


# Import libraries

In [18]:
import numpy as np
import tensorly as tl
from tensorly import unfold
from tensorly.tenalg import mode_dot
from typing import Set, Tuple

In [5]:
# Устанавливаем бэкэнд TensorLy на NumPy (по умолчанию)
tl.set_backend('numpy')

In [6]:
seed = 42

tl.check_random_state(seed)
np.random.seed(seed)

# Algoritm using article idea

In [36]:
# Размеры тензора
n1, n2, m = 5, 5, 5
N = 3  # Число итераций

# Создаем случайный тензор X
X = tl.tensor(np.random.rand(n1, n2, m), dtype=tl.float32)

In [37]:
# Инициализация
R = X.copy()
Q = tl.tensor(np.eye(m), dtype=tl.float32)
P = tl.zeros((n1, n2), dtype=tl.float32)
M = tl.zeros((n1, n2), dtype=tl.float32)
A = set()

## First variant

In [38]:
for d in range(N):
    # Вычисляем нормы трубчатых фибр и заполняем матрицу M
    for i in range(n1):
        for j in range(n2):
            tube = R[i, j, :]
            M[i, j] = tl.norm(tube, 1)
    
    # Находим максимальный элемент в M, который не был использован
    while True:
        max_index = tl.argmax(M)
        x, y = divmod(max_index.item(), n2)
        if (x, y) not in A:
            break
        else:
            M[x, y] = 0  # Обнуляем элемент, чтобы не выбирать его снова
    
    A.add((x, y))
    P[x, y] = 1  # Устанавливаем соответствующий элемент в P равным 1
    
    # Извлекаем вектор t
    t = R[x, y, d:]
    
    # Вычисляем σ и вектор u
    sigma = tl.norm(t, 2)
    if sigma == 0:
        u = tl.zeros_like(t)
    else:
        e1 = tl.zeros_like(t)
        e1[0] = 1
        sign_t1 = tl.sign(t[0]) if t[0] != 0 else 1
        u = t + sign_t1 * sigma * e1
        denominator = tl.sqrt(2 * sigma * (sigma + tl.abs(t[0])))
        u = u / denominator

    # Обновляем только выбранную трубчатую фибру R[x, y, d:]
    R[x, y, d:] = R[x, y, d:] - 2 * u * tl.dot(tl.transpose(u), R[x, y, d:])
    
    # Обновляем Q
    u_Q = u.reshape(-1, 1)  # Размерность (m - d, 1)
    Q_d = Q[:, d:]          # Размерность (m, m - d)
    Q[:, d:] = Q_d - 2 * Q_d @ (u_Q @ u_Q.T)

print("P:")
print(P)
print("\nQ:")
print(Q)
print("\nR:")
print(R)

P:
[[0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

Q:
[[-0.43123817  0.82567656 -0.3545223   0.08019531  0.01321003]
 [-0.53034306  0.06494915  0.8306248   0.15514696 -0.02257791]
 [-0.52870864 -0.30869108 -0.22212034 -0.5629372  -0.5088626 ]
 [-0.29465252 -0.36285633 -0.314533    0.7968999  -0.21802461]
 [-0.4079423  -0.29510164 -0.19002092 -0.13247766  0.83237004]]

R:
[[[ 6.98161721e-01  5.36096394e-01  3.09527606e-01  8.13795030e-01
    6.84731185e-01]
  [ 1.62616938e-01  9.10927176e-01 -1.45098448e+00  5.96046448e-08
    5.96046448e-08]
  [ 6.13415182e-01  4.18243051e-01  9.32728469e-01  8.66063893e-01
    4.52186689e-02]
  [ 2.63669752e-02  3.76463354e-01  8.10553312e-01  9.87276137e-01
    1.50416896e-01]
  [ 5.94130695e-01 -1.58043373e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]]

 [[ 4.68693167e-01  4.14819509e-01  2.73407072e-01  5.63754961e-02
    8.64722371e-01]
  [-1.88503957e+00  5.96046448e-08  5.96046448e-08  5.9604

In [39]:
Q_T_Q = tl.dot(Q.T, Q)
identity = tl.tensor(np.eye(m), dtype=tl.float32)
difference = tl.norm(Q_T_Q - identity)


print(f"\nNorm of (Q^T Q - I): {difference}")


Norm of (Q^T Q - I): 2.42633689140348e-07


In [40]:
# Восстановление X из R и Q
R_reshaped = R.reshape(n1 * n2, m)
X_approx = tl.dot(R_reshaped, Q.T)
X_approx = X_approx.reshape(n1, n2, m)

# Вычисление ошибки восстановления
reconstruction_error = tl.norm(X - X_approx) / tl.norm(X)
print("Ошибка восстановления:", reconstruction_error)

Ошибка восстановления: 1.4930887


## Second variant

In [41]:
# Initialization
R = X.copy()
Q = tl.tensor(np.eye(m), dtype=tl.float32)
P = tl.zeros((n1, n2), dtype=tl.float32)
M = tl.zeros((n1, n2), dtype=tl.float32)
A = set()

In [42]:
for d in range(N):
    # Вычисляем трубчатые ℓ1-нормы и заполняем матрицу M
    for i in range(n1):
        for j in range(n2):
            tube = R[i, j, :]
            M[i, j] = tl.norm(tube, 1)  # Используем ℓ1-норму

    # Находим максимальный элемент в M, который не был использован
    while True:
        max_index = tl.argmax(M)
        max_index = int(max_index)
        x, y = divmod(max_index, n2)
        if (x, y) not in A:
            break
        else:
            M[x, y] = 0  # Обнуляем элемент, чтобы не выбирать его снова

    A.add((x, y))
    P[x, y] = 1  # Устанавливаем соответствующий элемент в P равным 1

    # Извлекаем вектор t
    t = R[x, y, d:]

    # Вычисляем σ и вектор u
    sigma = tl.norm(t, 2)
    if sigma == 0:
        u = tl.zeros_like(t)
    else:
        e_d = tl.zeros_like(t)
        e_d[0] = 1  # Позиция 0 соответствует позиции d в Python
        t1 = t[0]
        sign_t1 = tl.sign(t1) if t1 != 0 else 1
        numerator = t + sign_t1 * sigma * e_d
        denominator = tl.sqrt(2 * sigma * (sigma + tl.abs(t1)))
        u = numerator / denominator

    # Обновляем R_x,y,d:m
    R_slice = R[x, y, d:]
    R[x, y, d:] = R_slice - 2 * u * tl.dot(u, R_slice)

    # Обновляем Q
    u_Q = u.reshape(-1, 1)  # Размерность (m - d, 1)
    Q_d = Q[:, d:]  # Размерность (m, m - d)
    Q[:, d:] = Q_d - 2 * Q_d @ (u_Q @ u_Q.T)

print("P:")
print(P)
print("\nQ:")
print(Q)
print("\nR:")
print(R)

P:
[[0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

Q:
[[-0.43123817  0.82567656 -0.3545223   0.08019531  0.01321003]
 [-0.53034306  0.06494915  0.8306248   0.15514696 -0.02257791]
 [-0.52870864 -0.30869108 -0.22212034 -0.5629372  -0.5088626 ]
 [-0.29465252 -0.36285633 -0.314533    0.7968999  -0.21802461]
 [-0.4079423  -0.29510164 -0.19002092 -0.13247766  0.83237004]]

R:
[[[ 6.98161721e-01  5.36096394e-01  3.09527606e-01  8.13795030e-01
    6.84731185e-01]
  [ 1.62616938e-01  9.10927176e-01 -1.45098448e+00  5.96046448e-08
    5.96046448e-08]
  [ 6.13415182e-01  4.18243051e-01  9.32728469e-01  8.66063893e-01
    4.52186689e-02]
  [ 2.63669752e-02  3.76463354e-01  8.10553312e-01  9.87276137e-01
    1.50416896e-01]
  [ 5.94130695e-01 -1.58043373e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]]

 [[ 4.68693167e-01  4.14819509e-01  2.73407072e-01  5.63754961e-02
    8.64722371e-01]
  [-1.88503957e+00  5.96046448e-08  5.96046448e-08  5.9604

In [43]:
Q_T_Q = tl.dot(Q.T, Q)
identity = tl.tensor(np.eye(m), dtype=tl.float32)
difference = tl.norm(Q_T_Q - identity)


print(f"\nNorm of (Q^T Q - I): {difference}")


Norm of (Q^T Q - I): 2.42633689140348e-07


In [44]:
# Восстановление X из R и Q
R_reshaped = R.reshape(n1 * n2, m)
X_approx = tl.dot(R_reshaped, Q.T)
X_approx = X_approx.reshape(n1, n2, m)

# Вычисление ошибки восстановления
reconstruction_error = tl.norm(X - X_approx) / tl.norm(X)
print("Ошибка восстановления:", reconstruction_error)

Ошибка восстановления: 1.4930887


## Final variant

In [45]:
def tensor_based_tube_fiber_pivot_qr_factorization(R: tl.tensor, N: int, A: Set[Tuple[int, int]], 
                                                   P: tl.tensor, Q: tl.tensor, M: tl.tensor) -> Tuple[tl.tensor, tl.tensor, tl.tensor]:
    """
    Implements the Tensor-based Tube Fiber-pivot QR Factorization.

    Parameters:
    - R: Input 3D tensor of shape (n1 x n2 x m), where n1 and n2 are dimensions of the matrix,
         and m is the size of each tube (depth).
    - N: Number of iterations for the factorization.
    - A: Set of already used indices (to avoid repetition in the pivot).
    - P: Output permutation matrix (n1 x n2) for selecting sensors or fibers.
    - Q: Orthogonal matrix to be updated (m x m).
    - M: Matrix to store the ℓ1-norms (n1 x n2) of the tubes from tensor R.

    Returns:
    - P: Updated permutation matrix with selected sensor placements.
    - Q: Updated orthogonal matrix after each iteration.
    - R: Updated tensor after applying Householder transformations.
    """

    n1, n2, _ = R.shape

    for d in range(N):
        # Compute tubular ℓ1-norms and fill matrix M
        for i in range(n1):
            for j in range(n2):
                tube = R[i, j, :]
                M[i, j] = tl.norm(tube, 1)  # Using ℓ1-norm

        # Find the maximum element in M that has not been used
        while True:
            max_index = tl.argmax(M)
            max_index = int(max_index)
            x, y = divmod(max_index, n2)
            if (x, y) not in A:
                break
            else:
                M[x, y] = 0  # Zero out the element to avoid reusing it

        A.add((x, y))
        P[x, y] = 1  # Set the corresponding element in P to 1

        # Extract vector t from tensor R
        t = R[x, y, d:]

        # Compute sigma and vector u
        sigma = tl.norm(t, 2)
        if sigma == 0:
            u = tl.zeros_like(t)
        else:
            e_d = tl.zeros_like(t)
            e_d[0] = 1  # Position 0 corresponds to position d in Python
            t1 = t[0]
            sign_t1 = tl.sign(t1) if t1 != 0 else 1
            numerator = t + sign_t1 * sigma * e_d
            denominator = tl.sqrt(2 * sigma * (sigma + tl.abs(t1)))
            u = numerator / denominator

        # Update R for slice x, y, d:m
        R_slice = R[x, y, d:]
        R[x, y, d:] = R_slice - 2 * u * tl.dot(u, R_slice)

        # Update Q matrix
        u_Q = u.reshape(-1, 1)  # Reshape u to (m - d, 1)
        Q_d = Q[:, d:]  # Get submatrix of Q from column d onwards
        Q[:, d:] = Q_d - 2 * Q_d @ (u_Q @ u_Q.T)  # Update Q

    # Return the updated P, Q, and R matrices
    return P, Q, R

In [46]:
# Initialization
R = X.copy()
Q = tl.tensor(np.eye(m), dtype=tl.float32)
P = tl.zeros((n1, n2), dtype=tl.float32)
M = tl.zeros((n1, n2), dtype=tl.float32)
A = set()

# Call the factorization function
P_updated, Q_updated, R_updated = tensor_based_tube_fiber_pivot_qr_factorization(R, N, A, P, Q, M)

# Now P_updated, Q_updated, and R_updated contain the outputs of the factorization

In [47]:
print("P:")
print(P_updated)
print("\nQ:")
print(Q_updated)
print("\nR:")
print(R_updated)

P:
[[0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

Q:
[[-0.43123817  0.82567656 -0.3545223   0.08019531  0.01321003]
 [-0.53034306  0.06494915  0.8306248   0.15514696 -0.02257791]
 [-0.52870864 -0.30869108 -0.22212034 -0.5629372  -0.5088626 ]
 [-0.29465252 -0.36285633 -0.314533    0.7968999  -0.21802461]
 [-0.4079423  -0.29510164 -0.19002092 -0.13247766  0.83237004]]

R:
[[[ 6.98161721e-01  5.36096394e-01  3.09527606e-01  8.13795030e-01
    6.84731185e-01]
  [ 1.62616938e-01  9.10927176e-01 -1.45098448e+00  5.96046448e-08
    5.96046448e-08]
  [ 6.13415182e-01  4.18243051e-01  9.32728469e-01  8.66063893e-01
    4.52186689e-02]
  [ 2.63669752e-02  3.76463354e-01  8.10553312e-01  9.87276137e-01
    1.50416896e-01]
  [ 5.94130695e-01 -1.58043373e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]]

 [[ 4.68693167e-01  4.14819509e-01  2.73407072e-01  5.63754961e-02
    8.64722371e-01]
  [-1.88503957e+00  5.96046448e-08  5.96046448e-08  5.9604